# Save predictions to Pandas

This notebook loads all 2x5 trained models, gets the predictions and saves the results to pandas dataframe and a file

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [4]:
## TODO: train a final CC and CW 3D CNN models with the updated data and do predictions and save to pandas csv gz
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [ ]:
import os
# do we have the correct env loaded?
assert 'donkey' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [6]:
![ ! -d "data/" ] && \
mkdir data && \
curl -L "https://www.dropbox.com/s/eltelvjt12a4ny1/data.zip?dl=1" --output data/data.zip && \
unzip data/data.zip -d data

In [7]:
![ ! -d "models/" ] && \
mkdir models && \
curl -L "https://www.dropbox.com/s/k9amvpqz3lcu7f3/models.zip?dl=1" --output models/models.zip && \
unzip models/models.zip -d models

In [8]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/


### Common settings for both directions

In [9]:
from dataloader import load_data
import numpy as np
import donkeycar as dk
from donkeycar.parts import keras
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

SEQUENCE_LENGTH = 3
cfg.SEQUENCE_LENGTH = SEQUENCE_LENGTH
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
N_FOLDS = 5
DATA_PATH = 'data'
MODELS_PATH = 'models'

MODEL_CLASS = keras.Keras3D_CNN_ModifiedOnlySteering


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py



### Load Counter Clockwise tubs and get predictions

In [10]:
from dataloader import get_folds
import itertools

DIRECTION = 'CC'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

from losses import ModelResults

CC_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-1-CC-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-1-CC-80']
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-1-CC-80/catalog_5.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-3-CC-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-3-CC-85/catalog_4.catalog


...

Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-3-CC-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-2-CC-90/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-2-CC-90/catalog_7.catalog


In [12]:
DIRECTION = 'CW'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

CW_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-1-CW-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-1-CW-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-3-CW-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-3-CW-85/catalog_6.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/3-3-CW-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/3-3-CW-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-2-CW-90']
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/3-2-CW-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-2-CW-90/catalog_2.catalog
Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/3-2-CW-90/catalog_6.catalog


In [13]:
CC_ModelResults.predict_results()

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
2022-03-29 12:09:39.389560: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-03-29 12:09:39.463011: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe0da5c2e90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-29 12:09:39.463023: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


Getting 90-speed  mse-s for fold 1
seq size 3
len ground_truth 4542
36/36 [==============================] - 36s 1s/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.16887981541865443
seq size 3
len ground_truth 5963
47/47 [==============================] - 53s 1s/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.1613558140436403
seq size 3
len ground_truth 2101
17/17 [==============================] - 5s 312ms/step
predictions shape (2101,)
test90 has 2101 predictions with an mse of 0.15109409898096268
seq size 3
len ground_truth 8404
66/66 [==============================] - 21s 316ms/step
predictions shape (8404,)
train90 has 8404 predictions with an mse of 0.16798764154531223
Getting 85-speed mse-s for fold 1
seq size 3
len ground_truth 4272
34/34 [==============================] - 36s 1s/step
predictions shape (4272,)
1-3-CC-85 has 4272 predictions with an mse of 0.14419504971619862
seq size 3
len ground_truth 4398
35/35 [==

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.12281019264172746
Getting 90-speed  mse-s for fold 2
seq size 3
len ground_truth 4542
36/36 [==============================] - 11s 306ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.10864025285896624
seq size 3
len ground_truth 5963
47/47 [==============================] - 14s 303ms/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.10729881489587496
seq size 3
len ground_truth 2101
17/17 [==============================] - 5s 296ms/step
predictions shape (2101,)
test90 has 2101 predictions with an mse of 0.12096296214606916
seq size 3
len ground_truth 8404
66/66 [==============================] - 20s 304ms/step
predictions shape (8404,)
train90 has 8404 predictions with an mse of 0.10460776748387718
Getting 85-speed mse-s for fold 2
seq size 3
len ground_truth 4272
34/34 [==============================] - 10s 301ms/step
predictions shape (4272,)
1-3-CC

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.08626317908254284
Getting 90-speed  mse-s for fold 3
seq size 3
len ground_truth 4542
36/36 [==============================] - 11s 296ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.09754230279398533
seq size 3
len ground_truth 5963
47/47 [==============================] - 14s 302ms/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.09670286274021844
seq size 3
len ground_truth 2101
17/17 [==============================] - 5s 289ms/step
predictions shape (2101,)
test90 has 2101 predictions with an mse of 0.10482325597004402
seq size 3
len ground_truth 8404
66/66 [==============================] - 20s 301ms/step
predictions shape (8404,)
train90 has 8404 predictions with an mse of 0.09512644602629013
Getting 85-speed mse-s for fold 3
seq size 3
len ground_truth 4272
34/34 [==============================] - 10s 300ms/step
predictions shape (4272,)
1-3-CC

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.08954331088777064
Getting 90-speed  mse-s for fold 4
seq size 3
len ground_truth 4542
36/36 [==============================] - 11s 301ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.14082064475764364
seq size 3
len ground_truth 5963
47/47 [==============================] - 14s 300ms/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.126213652285599
seq size 3
len ground_truth 2101
17/17 [==============================] - 5s 285ms/step
predictions shape (2101,)
test90 has 2101 predictions with an mse of 0.13042874173745037
seq size 3
len ground_truth 8404
66/66 [==============================] - 20s 303ms/step
predictions shape (8404,)
train90 has 8404 predictions with an mse of 0.1330543301701845
Getting 85-speed mse-s for fold 4
seq size 3
len ground_truth 4272
34/34 [==============================] - 10s 295ms/step
predictions shape (4272,)
1-3-CC-85

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.09886526492495065
Getting 90-speed  mse-s for fold 5
seq size 3
len ground_truth 4542
36/36 [==============================] - 11s 294ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.12563442354727414
seq size 3
len ground_truth 5963
47/47 [==============================] - 14s 298ms/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.1175474288751648
seq size 3
len ground_truth 2101
17/17 [==============================] - 5s 287ms/step
predictions shape (2101,)
test90 has 2101 predictions with an mse of 0.1341893908711764
seq size 3
len ground_truth 8404
66/66 [==============================] - 20s 301ms/step
predictions shape (8404,)
train90 has 8404 predictions with an mse of 0.11775761111788745
Getting 85-speed mse-s for fold 5
seq size 3
len ground_truth 4272
34/34 [==============================] - 10s 296ms/step
predictions shape (4272,)
1-3-CC-8

In [14]:
CW_ModelResults.predict_results()

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CW.h5


Getting 90-speed  mse-s for fold 1
seq size 3
len ground_truth 1235
10/10 [==============================] - 9s 903ms/step
predictions shape (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.08435376705453354
seq size 3
len ground_truth 5217
41/41 [==============================] - 45s 1s/step
predictions shape (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.078962444431826
seq size 3
len ground_truth 1291
11/11 [==============================] - 3s 277ms/step
predictions shape (1291,)
test90 has 1291 predictions with an mse of 0.09765198712999218
seq size 3
len ground_truth 5161
41/41 [==============================] - 12s 294ms/step
predictions shape (5161,)
train90 has 5161 predictions with an mse of 0.07557745752804611
Getting 85-speed mse-s for fold 1
seq size 3
len ground_truth 6086
48/48 [==============================] - 50s 1s/step
predictions shape (6086,)
2-3-CW-85 has 6086 predictions with an mse of 0.07779715785824406
seq size 3
len ground_truth 4006
32/32 [=

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CW.h5


predictions shape (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.08078958128502264
Getting 90-speed  mse-s for fold 2
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 285ms/step
predictions shape (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.06983098143217648
seq size 3
len ground_truth 5217
41/41 [==============================] - 12s 302ms/step
predictions shape (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.07099302410883564
seq size 3
len ground_truth 1291
11/11 [==============================] - 3s 284ms/step
predictions shape (1291,)
test90 has 1291 predictions with an mse of 0.08234434901927873
seq size 3
len ground_truth 5161
41/41 [==============================] - 12s 299ms/step
predictions shape (5161,)
train90 has 5161 predictions with an mse of 0.06787547282602917
Getting 85-speed mse-s for fold 2
seq size 3
len ground_truth 6086
48/48 [==============================] - 14s 301ms/step
predictions shape (6086,)
2-3-CW-

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CW.h5


predictions shape (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.062494059534124825
Getting 90-speed  mse-s for fold 3
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 274ms/step
predictions shape (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.08251379469885683
seq size 3
len ground_truth 5217
41/41 [==============================] - 12s 296ms/step
predictions shape (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.07569809350314394
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 280ms/step
predictions shape (1290,)
test90 has 1290 predictions with an mse of 0.08123486266440205
seq size 3
len ground_truth 5162
41/41 [==============================] - 12s 299ms/step
predictions shape (5162,)
train90 has 5162 predictions with an mse of 0.07594508266202024
Getting 85-speed mse-s for fold 3
seq size 3
len ground_truth 6086
48/48 [==============================] - 14s 302ms/step
predictions shape (6086,)
2-3-CW

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CW.h5


predictions shape (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.07785909163980269
Getting 90-speed  mse-s for fold 4
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 275ms/step
predictions shape (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.07376947918766343
seq size 3
len ground_truth 5217
41/41 [==============================] - 12s 299ms/step
predictions shape (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.08132858891851887
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 285ms/step
predictions shape (1290,)
test90 has 1290 predictions with an mse of 0.09036894520833046
seq size 3
len ground_truth 5162
41/41 [==============================] - 12s 297ms/step
predictions shape (5162,)
train90 has 5162 predictions with an mse of 0.07726087094399164
Getting 85-speed mse-s for fold 4
seq size 3
len ground_truth 6086
48/48 [==============================] - 14s 297ms/step
predictions shape (6086,)
2-3-CW-

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CW.h5


predictions shape (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.07345078877814443
Getting 90-speed  mse-s for fold 5
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 275ms/step
predictions shape (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.07378964571720352
seq size 3
len ground_truth 5217
41/41 [==============================] - 12s 298ms/step
predictions shape (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.07486413790698151
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 277ms/step
predictions shape (1290,)
test90 has 1290 predictions with an mse of 0.08613030260183685
seq size 3
len ground_truth 5162
41/41 [==============================] - 12s 297ms/step
predictions shape (5162,)
train90 has 5162 predictions with an mse of 0.07179161762065804
Getting 85-speed mse-s for fold 5
seq size 3
len ground_truth 6086
48/48 [==============================] - 14s 298ms/step
predictions shape (6086,)
2-3-CW-

In [15]:

results_unpacked = []

for fold, fold_results in CC_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

for fold, fold_results in CW_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

len(results_unpacked)

324050

In [16]:
import pandas as pd
df = pd.DataFrame(results_unpacked, columns=['name', 'direction', 'speed', 'fold', 'is_train', 'predicted_angle', 'ground_truth_angle'])


In [17]:
df.to_csv('3D-CNN-model-results.csv.gz', index=False)